In [146]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
import csv
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.decomposition import LatentDirichletAllocation

df = pd.read_csv('../dados_categorizados_completo_novo.csv')
df.head()

,Id,Enunciado,Tópico,Contexto
0,994,Meu primeiro programa\nEscreva um programa que...,Ambientação,Outros
1,996,Impressão de caracteres na tela\nEscreva um pr...,Ambientação,Outros
2,999,Impressão de caracteres na tela (Bart Simpson)...,Ambientação,Outros
3,1000,Operadores aritméticos\nQual o valor de X para...,Ambientação,Matemático
4,578,Operadores aritméticos \nQual o valor de Y par...,Ambientação,Matemático


In [27]:
df.Contexto.value_counts()

Matemático         261
Comercial          120
Jogos               96
Escolar             79
Outros              59
Trânsito            43
Esporte             42
Física              36
Bancário            35
RH                  35
Filmes e séries     30
População           25
Química             23
Pessoa              22
Data e hora         21
Segurança           20
Pesquisa            18
Meio ambiente       18
Saúde               17
Consumo             16
Geografia           11
Produção             7
Computacional        6
Imposto              5
Name: Contexto, dtype: int64

In [28]:
import spacy
nlp = spacy.load('pt')

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
X = df['Enunciado']
y = df['Contexto']

In [147]:
class LemmaTokenizer(object):
    def __call__(self, text):
        doc=nlp(text)
        tokens=[]
        for token in doc:
            if(token.text.isalpha() and len(token.text)>=2):
                tokens+=[token]
        return [t.lemma_ for t in tokens if t not in nlp.Defaults.stop_words]
pipeline= LinearSVC(
    penalty='l2',
    loss='squared_hinge',
    dual=True,
    tol=0.001,
    C=10.0,
    multi_class='ovr',
    fit_intercept=True,
    intercept_scaling=10,
    class_weight=None,
    verbose=0,
    random_state=None,
    max_iter=10000,
)

In [31]:
text_clf_lsvc2 = Pipeline([('tfidf', TfidfVectorizer(stop_words=nlp.Defaults.stop_words,
                            tokenizer=LemmaTokenizer(),sublinear_tf=True)),
                            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)),
                            ('clf',pipeline),
    ])
text_clf_lsvc2.fit(X, y)

/home/samuel/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['acercar', 'algum', 'antar', 'apoiar', 'baixar', 'caminhar', 'ceder', 'cincar', 'comer', 'conhecer', 'custar', 'dever', 'devir', 'direito', 'entrar', 'estevar', 'estivar', 'falto', 'formar', 'grupar', 'irar', 'ligar', 'mear', 'nado', 'nenhum', 'nó', 'o', 'obrigar', 'oitavar', 'orar', 'outro', 'parecer', 'parir', 'pelar', 'pontar', 'pôr', 'quantum', 'quartar', 'querer', 'quietar', 'quintar', 'segundar', 'seriar', 'sobrar', 'suar', 'tardar', 'terceirar', 'umar', 'vezar', 'vir', 'vário'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/samuel/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={'acerca', 'ademais', 'adeus',
                                             'agora', 'ainda', 'algo',
                                             'alguma...
                                 tokenizer=<__main__.LemmaTokenizer object at 0x7fe5681b99d0>,
                                 use_idf=True, vocabulary=None)),
                ('svd',
                 TruncatedSVD(algorithm='randomized', n_components=300,
                  

In [148]:
def get_new_sentences(da,maximum,synonyms_lexicon):
    strings=[]
    for elemento in da['Enunciado']:
        if(len(da)+len(strings)<maximum):
            contador=0
            lista=elemento.split()
            while(contador<len(lista)):
                if(lista[contador] in synonyms_lexicon):
                    lista[contador]=synonyms_lexicon[lista[contador]][0]
                contador+=1
            strings+=[' '.join(lista)]
        else:
            break

    new_sentences = pd.Series(strings)
    return new_sentences

def get_synonyms_lexicon(path):
    synonyms_lexicon = {}
    text_entries = [l.strip() for l in open(path).readlines()]
    for e in text_entries:
        e = e.split(' ')
        k = e[0]
        v = e[1:len(e)]
        synonyms_lexicon[k] = v
    return synonyms_lexicon

synonyms_lexicon = get_synonyms_lexicon('../paráfrase/arq3.txt')

In [149]:
maximum = df.Contexto.value_counts().max()
g = df.groupby('Contexto')


mat = g.get_group('Matemático')
comercial = g.get_group('Comercial')
jogos = g.get_group('Jogos')
escolar = g.get_group('Escolar')
outros = g.get_group('Outros')
transito = g.get_group('Trânsito')
esporte = g.get_group('Esporte')
fisica = g.get_group('Física')
bancario = g.get_group('Bancário')
RH = g.get_group('RH')
f_s = g.get_group('Filmes e séries')
populacao = g.get_group('População')
quimica = g.get_group('Química')
pessoa = g.get_group('Pessoa')
d_h = g.get_group('Data e hora')
seguranca = g.get_group('Segurança')
ambiente = g.get_group('Meio ambiente')
pesquisa = g.get_group('Pesquisa')
saude = g.get_group('Saúde')
consumo = g.get_group('Consumo')
geografia = g.get_group('Geografia')
producao = g.get_group('Produção')
imposto = g.get_group('Imposto')
computacional = g.get_group('Computacional')

new_jogos = get_new_sentences(jogos,maximum,synonyms_lexicon)
new_comercial = get_new_sentences(comercial,maximum,synonyms_lexicon)
new_escolar = get_new_sentences(escolar,maximum,synonyms_lexicon)
new_outros = get_new_sentences(outros,maximum,synonyms_lexicon)
new_transito = get_new_sentences(transito,maximum,synonyms_lexicon)
new_esporte = get_new_sentences(esporte,maximum,synonyms_lexicon)
new_fisica = get_new_sentences(fisica,maximum,synonyms_lexicon)
new_bancario = get_new_sentences(bancario,maximum,synonyms_lexicon)
new_RH = get_new_sentences(RH,maximum,synonyms_lexicon)
new_f_s = get_new_sentences(f_s,maximum,synonyms_lexicon)
new_populacao = get_new_sentences(populacao,maximum,synonyms_lexicon)
new_quimica = get_new_sentences(quimica,maximum,synonyms_lexicon)
new_pessoa = get_new_sentences(pessoa,maximum,synonyms_lexicon)
new_d_h = get_new_sentences(d_h,maximum,synonyms_lexicon)
new_seguranca = get_new_sentences(seguranca,maximum,synonyms_lexicon)
new_pesquisa = get_new_sentences(pesquisa,maximum,synonyms_lexicon)
new_ambiente = get_new_sentences(ambiente,maximum,synonyms_lexicon)
new_saude = get_new_sentences(saude,maximum,synonyms_lexicon)
new_consumo = get_new_sentences(consumo,maximum,synonyms_lexicon)
new_geografia = get_new_sentences(geografia,maximum,synonyms_lexicon)
new_producao = get_new_sentences(producao,maximum,synonyms_lexicon)
new_computacional = get_new_sentences(computacional,maximum,synonyms_lexicon)
new_imposto = get_new_sentences(imposto,maximum,synonyms_lexicon)


new_jogos = pd.DataFrame(new_jogos,columns=['Enunciado'])
new_jogos['Contexto'] = 'Jogos'
new_jogos['Tópico']='Nada'

new_comercial = pd.DataFrame(new_comercial,columns=['Enunciado'])
new_comercial['Contexto'] = 'Comercial'
new_comercial['Tópico']='Nada'


new_escolar = pd.DataFrame(new_escolar,columns=['Enunciado'])
new_escolar['Contexto'] = 'Escolar'
new_escolar['Tópico']='Nada'


new_outros = pd.DataFrame(new_outros,columns=['Enunciado'])
new_outros['Contexto'] = 'Outros'
new_outros['Tópico']='Nada'


new_transito = pd.DataFrame(new_transito,columns=['Enunciado'])
new_transito['Contexto'] = 'Trânsito'
new_transito['Tópico']='Nada'


new_esporte = pd.DataFrame(new_esporte,columns=['Enunciado'])
new_esporte['Contexto'] = 'Esporte'
new_esporte['Tópico']='Nada'


new_fisica = pd.DataFrame(new_fisica,columns=['Enunciado'])
new_fisica['Contexto'] = 'Física'
new_fisica['Tópico']='Nada'


new_bancario = pd.DataFrame(new_bancario,columns=['Enunciado'])
new_bancario['Contexto'] = 'Bancário'
new_bancario['Tópico']='Nada'


new_RH = pd.DataFrame(new_RH,columns=['Enunciado'])
new_RH['Contexto'] = 'RH'
new_RH['Tópico']='Nada'


new_f_s = pd.DataFrame(new_f_s,columns=['Enunciado'])
new_f_s['Contexto'] = 'Filmes e séries'
new_f_s['Tópico']='Nada'


new_populacao = pd.DataFrame(new_populacao,columns=['Enunciado'])
new_populacao['Contexto'] = 'População'
new_populacao['Tópico']='Nada'


new_quimica = pd.DataFrame(new_quimica,columns=['Enunciado'])
new_quimica['Contexto'] = 'Química'
new_quimica['Tópico']='Nada'


new_pessoa = pd.DataFrame(new_pessoa,columns=['Enunciado'])
new_pessoa['Contexto'] = 'Pessoa'
new_pessoa['Tópico']='Nada'


new_d_h = pd.DataFrame(new_d_h,columns=['Enunciado'])
new_d_h['Contexto'] = 'Data e hora'
new_d_h['Tópico']='Nada'


new_seguranca = pd.DataFrame(new_seguranca,columns=['Enunciado'])
new_seguranca['Contexto'] = 'Segurança'
new_seguranca['Tópico']='Nada'


new_pesquisa = pd.DataFrame(new_pesquisa,columns=['Enunciado'])
new_pesquisa['Contexto'] = 'Pesquisa'
new_pesquisa['Tópico']='Nada'


new_ambiente = pd.DataFrame(new_ambiente,columns=['Enunciado'])
new_ambiente['Contexto'] = 'Meio ambiente'
new_ambiente['Tópico']='Nada'


new_saude = pd.DataFrame(new_saude,columns=['Enunciado'])
new_saude['Contexto'] = 'Saúde'
new_saude['Tópico']='Nada'


new_consumo = pd.DataFrame(new_consumo,columns=['Enunciado'])
new_consumo['Contexto'] = 'Consumo'
new_consumo['Tópico']='Nada'


new_geografia = pd.DataFrame(new_geografia,columns=['Enunciado'])
new_geografia['Contexto'] = 'Geografia'
new_geografia['Tópico']='Nada'


new_producao = pd.DataFrame(new_producao,columns=['Enunciado'])
new_producao['Contexto'] = 'Produção'
new_producao['Tópico']='Nada'


new_computacional= pd.DataFrame(new_computacional,columns=['Enunciado'])
new_computacional['Contexto'] = 'Computacional'
new_computacional['Tópico']='Nada'


new_imposto= pd.DataFrame(new_imposto,columns=['Enunciado'])
new_imposto['Contexto'] = 'Imposto'
new_imposto['Tópico']='Nada'


jogos_extended = pd.concat([jogos, new_jogos], keys = ['Enunciado', 'Contexto'])
comercial_extended = pd.concat([comercial, new_comercial], keys = ['Enunciado', 'Contexto'])
escolar_extended = pd.concat([escolar, new_escolar], keys = ['Enunciado', 'Contexto'])
outros_extended = pd.concat([outros, new_outros], keys = ['Enunciado', 'Contexto'])
transito_extended = pd.concat([transito, new_transito], keys = ['Enunciado', 'Contexto'])
esporte_extended = pd.concat([esporte, new_esporte], keys = ['Enunciado', 'Contexto'])
fisica_extended = pd.concat([fisica, new_fisica], keys = ['Enunciado', 'Contexto'])
bancario_extended = pd.concat([bancario, new_bancario], keys = ['Enunciado', 'Contexto'])
RH_extended = pd.concat([RH, new_RH], keys = ['Enunciado', 'Contexto'])
f_s_extended = pd.concat([f_s, new_f_s], keys = ['Enunciado', 'Contexto'])
populacao_extended = pd.concat([populacao, new_populacao], keys = ['Enunciado', 'Contexto'])
quimica_extended = pd.concat([quimica, new_quimica], keys = ['Enunciado', 'Contexto'])
pessoa_extended = pd.concat([pessoa, new_pessoa], keys = ['Enunciado', 'Contexto'])
d_h_extended = pd.concat([d_h, new_d_h], keys = ['Enunciado', 'Contexto'])
seguranca_extended = pd.concat([seguranca, new_seguranca], keys = ['Enunciado', 'Contexto'])
pesquisa_extended = pd.concat([pesquisa, new_pesquisa], keys = ['Enunciado', 'Contexto'])
ambiente_extended = pd.concat([ambiente, new_ambiente], keys = ['Enunciado', 'Contexto'])
saude_extended = pd.concat([saude, new_saude], keys = ['Enunciado', 'Contexto'])
consumo_extended = pd.concat([consumo, new_consumo], keys = ['Enunciado', 'Contexto'])
geografia_extended = pd.concat([geografia, new_geografia], keys = ['Enunciado', 'Contexto'])
producao_extended = pd.concat([producao, new_producao], keys = ['Enunciado', 'Contexto'])
computacional_extended = pd.concat([computacional, new_computacional], keys = ['Enunciado', 'Contexto'])
imposto_extended = pd.concat([imposto, new_imposto], keys = ['Enunciado', 'Contexto'])

df = pd.concat([mat, jogos_extended,comercial_extended,escolar_extended,outros_extended,
           transito_extended,esporte_extended,fisica_extended,bancario_extended,
           RH_extended,f_s_extended,populacao_extended,quimica_extended,
           pessoa_extended,d_h_extended,seguranca_extended,pesquisa_extended,
           ambiente_extended,saude_extended,consumo_extended,geografia_extended,
           producao_extended,computacional_extended,imposto_extended])

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:169: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:170: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:171: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 

In [150]:
df.Contexto.value_counts()

Matemático         261
Comercial          240
Jogos              192
Escolar            158
Outros             118
Trânsito            86
Esporte             84
Física              72
RH                  70
Bancário            70
Filmes e séries     60
População           50
Química             46
Pessoa              44
Data e hora         42
Segurança           40
Pesquisa            36
Meio ambiente       36
Saúde               34
Consumo             32
Geografia           22
Produção            14
Computacional       12
Imposto             10
Name: Contexto, dtype: int64

In [151]:
cv=CountVectorizer(max_df=0.5,min_df=5,stop_words=nlp.Defaults.stop_words)
dtm=cv.fit_transform(df['Enunciado'])
LDA=LatentDirichletAllocation(n_components=1100,random_state=42)
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=1100, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [152]:
teste=[]
for i,topic in enumerate(LDA.components_):
    teste+=[' '.join([cv.get_feature_names()[index] for index in topic.argsort()[-25:]])]
predictions=text_clf_lsvc2.predict(teste)

In [153]:
predictions

array(['Comercial', 'Comercial', 'Comercial', ..., 'Comercial',
       'Comercial', 'Comercial'], dtype=object)

In [154]:
topic_results=LDA.transform(dtm)

In [155]:
y_pred=[]
for elemento in topic_results.argmax(axis=1):
    y_pred+=[predictions[elemento]]


In [156]:
df['LDA']=y_pred

In [157]:
df.LDA.value_counts()

Comercial          334
Matemático         310
Jogos              189
Escolar            134
Esporte             97
Outros              94
Trânsito            75
Física              65
RH                  63
População           56
Bancário            56
Filmes e séries     55
Segurança           42
Data e hora         39
Pessoa              36
Química             36
Pesquisa            35
Meio ambiente       34
Consumo             28
Saúde               19
Geografia           13
Imposto              8
Produção             7
Computacional        4
Name: LDA, dtype: int64

In [158]:
df.Contexto.value_counts()

Matemático         261
Comercial          240
Jogos              192
Escolar            158
Outros             118
Trânsito            86
Esporte             84
Física              72
RH                  70
Bancário            70
Filmes e séries     60
População           50
Química             46
Pessoa              44
Data e hora         42
Segurança           40
Pesquisa            36
Meio ambiente       36
Saúde               34
Consumo             32
Geografia           22
Produção            14
Computacional       12
Imposto             10
Name: Contexto, dtype: int64

In [159]:
df

,Contexto,Enunciado,Id,Tópico,LDA
3,Matemático,Operadores aritméticos\nQual o valor de X para...,1000.0,Ambientação,Outros
4,Matemático,Operadores aritméticos \nQual o valor de Y par...,578.0,Ambientação,Comercial
6,Matemático,Arredondamento\nEscreva um programa que aprese...,797.0,Ambientação,Matemático
12,Matemático,Ordem crescente\nEscreva um programa que leia ...,1015.0,Estrutura condicional composta,Matemático
13,Matemático,Área do Círculo e Volume da Esfera\nEscreva um...,825.0,Variáveis e Estrutura Sequencial,Trânsito
...,...,...,...,...,...
"(Contexto, 0)",Imposto,Impostos Astrobaldo pormenorizadas comprada ba...,NaN,Nada,Imposto
"(Contexto, 1)",Imposto,Impostos Astrobaldo pormenorizadas comprada ba...,NaN,Nada,Imposto
"(Contexto, 2)",Imposto,Impostos Fredegunda pormenorizadas comprada ba...,NaN,Nada,Imposto
"(Contexto, 3)",Imposto,Impostos Fredegunda pormenorizadas comprada ba...,NaN,Nada,Imposto


In [160]:
contexto=[]
for elemento in df['Contexto']:
    contexto+=[elemento]
lda=[]
for elemento in df['LDA']:
    lda+=[elemento]
topico=[]
for elemento in df['Tópico']:
    topico+=[elemento]

In [161]:
contador=0
y_test=[]
y_pred=[]
while(contador<len(contexto)):
    if(topico[contador]!='Nada'):
        y_test+=[contexto[contador]]
        y_pred+=[lda[contador]]
    contador+=1

In [162]:
print(metrics.confusion_matrix(y_test,y_pred))

[[ 31   2   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   1
    0   0   0   0   0   0]
 [  2 108   0   1   0   0   2   1   0   0   0   1   4   0   0   0   0   0
    1   0   0   0   0   0]
 [  0   2   4   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0  16   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0  21   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0  73   1   0   0   0   0   1   2   0   2   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0  38   0   0   0   0   0   0   0   3   1   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  29   0   0   0   0   0   0   1   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   1   0   0   0  32   0   0   3   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   2   0   0   0   0   0   0   0   6   0   0   0   0   3   0   0   0
    0   0   0   0

In [163]:
print(metrics.classification_report(y_test,y_pred))

                 precision    recall  f1-score   support

       Bancário       0.89      0.89      0.89        35
      Comercial       0.79      0.90      0.84       120
  Computacional       1.00      0.67      0.80         6
        Consumo       0.94      1.00      0.97        16
    Data e hora       0.84      1.00      0.91        21
        Escolar       0.96      0.92      0.94        79
        Esporte       0.93      0.90      0.92        42
Filmes e séries       0.91      0.97      0.94        30
         Física       1.00      0.89      0.94        36
      Geografia       1.00      0.55      0.71        11
        Imposto       1.00      0.80      0.89         5
          Jogos       0.89      0.94      0.91        96
     Matemático       0.95      0.91      0.93       261
  Meio ambiente       1.00      1.00      1.00        18
         Outros       0.74      0.78      0.76        59
       Pesquisa       0.90      1.00      0.95        18
         Pessoa       0.95    

In [164]:
print(metrics.accuracy_score(y_test,y_pred))

0.9090909090909091
